# Conformal prediction for Regressors

In [9]:
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [5]:
housing = fetch_california_housing(data_home='data/california_housing')
print(housing.data.shape, housing.target.shape)
print(housing.feature_names)

(20640, 8) (20640,)
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [12]:
# Treure random_state

In [10]:
X, y = housing.data, housing.target
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [16]:
learning_rate = 0.1
n_estimators = 200

In [17]:
loss = 'quantile'

In [30]:
upper_alpha = 0.95
lower_alpha = 1-upper_alpha
print("Coverage: %.2f" % (upper_alpha-lower_alpha))

Coverage: 0.90


lq = lower quantile\
uq = upper quantile

In [25]:
gbr_lq = GradientBoostingRegressor(loss=loss, learning_rate=learning_rate, n_estimators=n_estimators, alpha=lower_alpha)
gbr_uq = GradientBoostingRegressor(loss=loss, learning_rate=learning_rate, n_estimators=n_estimators, alpha=upper_alpha)

gbr_lq.fit(X_train, y_train)
gbr_uq.fit(X_train, y_train)

print("Done")

Done


200